In [1]:
import cv2
import json
import random
import imgaug.augmenters as iaa
from pathlib import Path

In [2]:
from simple_copy_paste import SimpleCopyPaste
copy_paste = SimpleCopyPaste(
    background_dir='./sample/back_ground/',
    template_ratio=random.uniform(0.4, 0.8),
    template_type='CARD_BACK',
    image_pattern='*.*',
    template_transforms=[
        iaa.Affine(rotate=(-20, 20), shear=(-15, 15), fit_output=True),
        iaa.Rot90(k=[1, 2, 3], keep_size=False),
        iaa.PerspectiveTransform(scale=(0.01, 0.03), keep_size=False, fit_output=True),
        iaa.ElasticTransformation(alpha=(0, 0.5), sigma=0.25)
    ],
    background_transforms=[
        iaa.ElasticTransformation(alpha=(0.1, 0.5), sigma=0.25),
        iaa.ChangeColorTemperature(kelvin=(4000, 9000)),
    ],
    final_transforms=[
        iaa.ElasticTransformation(alpha=(0.1, 0.5), sigma=0.25),
        iaa.ChangeColorTemperature(kelvin=(4000, 9000)),
    ]
)

In [3]:
label_paths = list(Path('./sample/fore_ground/COMMUNIST_PARTY/CARD_BACK/').glob('**/*.json'))

output_dir = Path('./sample/copy_paste/')
if not output_dir.exists():
    output_dir.mkdir(parents=True)

In [5]:
for i in range(5):
    for label_path in label_paths:
        idx = random.randint(1, 1000)

        with label_path.open(mode='r', encoding='utf-8') as f:
            label = json.load(f)

        image_name = Path(label['imagePath']).name
        image = cv2.imread(str(label_path.with_name(image_name)))

        image, label = copy_paste(image, label)
        
        image_path = output_dir.joinpath(f'{label_path.stem}_{idx}{Path(image_name).suffix}')
        json_path = output_dir.joinpath(f'{label_path.stem}_{idx}.json')
        
        label['imagePath'] = image_path.name
        with json_path.open(mode='w', encoding='utf-8') as f:
            json.dump(label, f, indent=4, ensure_ascii=False)

        cv2.imwrite(str(image_path), image)